In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

### create model class

In [3]:
class NeuralNetwork(torch.nn.Module):
    # parameterize num of inputs & outputs to reuse same code for diff datasets with diff num of features and classes
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # first hidden layer
            # Linear layer takes num of input and output nodes as args
            torch.nn.Linear(num_inputs, 30),
            # Nonlinear activation functions are placed bw hidden layers
            torch.nn.ReLU(),

            # second hidden layer
            # The num of output nodes in prev hidden layer is equal to input of next hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        # the output of the last layer are called logits
        return logits

### create ToyDataset

In [4]:
# create Toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [5]:
# create custome dataset class
class ToyDataset(Dataset):
    def __init__(self, X, y):
        # these vars could be file paths, file objects, database connectors, etc. we are using X,y bc created toydataset above sitting in mem
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        # retrieve exactly one data record and corresponding label
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        # return dataset row len
        return self.labels.shape[0]

In [6]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

### create DataLoader

In [7]:
torch.manual_seed(123)

In [8]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

### train NN Model with GPU

In [16]:
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# NEW GPU training code
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
model = model.to(device)

optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)

In [17]:
device

# mps bc no GPU on macbook pro :(

device(type='mps')

In [18]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        # transfers data onto GPU
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")
        
    model.eval()

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.96
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 3.19
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.32
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 2.24
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.57
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.16
